# Set up config variables

In [1]:
import json

In [2]:
latMin  = -65
latMax  = -60
longMin = -70
longMax    = -50
startTime  = "2019-01-01"
endTime    = "2019-01-14"
cellWidth  = 5
cellHeight = 2.5

config = ' {"Mesh_info" : {"Region": {} } }'
config  = json.loads(config)

In [3]:
config['Mesh_info']['Region']['latMin'] = latMin
config['Mesh_info']['Region']['latMax'] = latMax
config['Mesh_info']['Region']['longMin'] = longMin
config['Mesh_info']['Region']['longMax'] = longMax
config['Mesh_info']['Region']['startTime'] = startTime
config['Mesh_info']['Region']['endTime'] = endTime
config['Mesh_info']['Region']['cellWidth'] = cellWidth
config['Mesh_info']['Region']['cellHeight'] = cellHeight

config

{'Mesh_info': {'Region': {'latMin': -65,
   'latMax': -60,
   'longMin': -70,
   'longMax': -50,
   'startTime': '2019-01-01',
   'endTime': '2019-01-14',
   'cellWidth': 5,
   'cellHeight': 2.5}}}

### Generate empty mesh, just with config data

In [4]:
from polar_route.mesh import Mesh

demo_mesh = Mesh(config)

ImportError: dlopen(/Users/ayat/BAS/bas-venv/lib/python3.10/site-packages/pyproj/_network.cpython-310-darwin.so, 2): Symbol not found: ____chkstk_darwin
  Referenced from: /Users/ayat/BAS/bas-venv/lib/python3.10/site-packages/pyproj/.dylibs/libtiff.5.dylib (which was built for Mac OS X 11.0)
  Expected in: /usr/lib/libSystem.B.dylib
 in /Users/ayat/BAS/bas-venv/lib/python3.10/site-packages/pyproj/.dylibs/libtiff.5.dylib

# Use Data Loaders for each data type

In [ ]:
import polar_route.data_loaders as data_loader

SIC_loader = getattr(data_loader, "load_amsr_folder")
params = dict()
params['folder'] = './datastore/sic/amsr_south/'
sic_datapoints = SIC_loader(params, longMin,longMax,latMin,latMax,startTime,endTime)

In [ ]:
gebco_loader = getattr(data_loader, "load_gebco")
params = dict()
params['file'] = './DEPLOYMENT/datastore/bathymetry/GEBCO/gebco_2022_n-40.0_s-90.0_w-140.0_e0.0.nc'
params['downsample_factors'] = [5,5]

bathymetry_datapoints = gebco_loader(params, longMin,longMax,latMin,latMax,startTime,endTime)

In [ ]:
current_loader = getattr(data_loader, "load_sose_currents")
params = dict()
params['file'] = './datastore/currents/sose_currents/SOSE_surface_velocity_6yearMean_2005-2010.nc'
current_datapoints = current_loader(params, longMin,longMax,latMin,latMax,startTime,endTime)


In [ ]:
thickness_loader = getattr(data_loader, "load_thickness")
params = dict()
# params['file'] = './datastore/currents/sose_currents/SOSE_surface_velocity_6yearMean_2005-2010.nc'
thickness_datapoints = thickness_loader(params, longMin,longMax,latMin,latMax,startTime,endTime)


In [ ]:
density_loader = getattr(data_loader, "load_density")
params = dict()
# params['file'] = './datastore/currents/sose_currents/SOSE_surface_velocity_6yearMean_2005-2010.nc'
density_datapoints = density_loader(params, longMin,longMax,latMin,latMax,startTime,endTime)


In [ ]:
bathymetry_datapoints

,lat,long,elevation
8067360,-64.997917,-69.997917,-3269
8067361,-64.997917,-69.977083,-3292
8067362,-64.997917,-69.956250,-3336
8067363,-64.997917,-69.935417,-3343
8067364,-64.997917,-69.914583,-3338
...,...,...,...
9674395,-60.018750,-50.102083,-3155
9674396,-60.018750,-50.081250,-3176
9674397,-60.018750,-50.060417,-3188
9674398,-60.018750,-50.039583,-3205


### Add datapoints to the mesh

In [ ]:
demo_mesh.add_data_points(sic_datapoints)
demo_mesh.add_data_points(bathymetry_datapoints)
demo_mesh.add_data_points(current_datapoints)
demo_mesh.add_data_points(thickness_datapoints)
demo_mesh.add_data_points(density_datapoints)

# Adjust methods of binning

Can be mean, max, min, std, etc... (some not implemented). Default is mean

e.g. For each cell, mean elevation is not as important as maximum elevation (ship could crash if not using max), so set MAX to be the output type for each cellbox

In [ ]:
value_output_type = '{"elevation": "MAX"}'
value_output_type = json.loads(value_output_type)

### Set data source if no datapoints remain after splitting

Can also be set to static value

e.g. If cellbox is split, and no datapoints remain, will default to NaN, which breaks code. 'parent' sets value to be same as parent cellbox if splitting and no datapoints remain. 

In [ ]:
value_fill_type = '{}'
value_fill_type = json.loads(value_fill_type)
value_fill_type['SIC'] = 'parent'
value_fill_type['uC'] = 'parent'
value_fill_type['vC'] = 'parent'
value_fill_type['thickness'] = 'parent'
value_fill_type['density'] = 'parent'
value_fill_type

{'SIC': 'parent',
 'uC': 'parent',
 'vC': 'parent',
 'thickness': 'parent',
 'density': 'parent'}

# Set cellbox behaviour metadata

5Setting output_type and fill_type first, as splitting operations will depend on these

In [ ]:
from polar_route.cellbox import CellBox

for cellbox in demo_mesh.cellboxes:
    if isinstance(cellbox, CellBox):
        cellbox.add_value_output_type(value_output_type)
        cellbox.set_value_fill_types(value_fill_type)
        

### Split based on each variable's extreme conditions

In [ ]:
splitting_condition = '{"elevation":{"threshold":-10,"upperBound": 1,"lowerBound":0}}'
splitting_condition = json.loads(splitting_condition)


for cellbox in demo_mesh.cellboxes:
    if isinstance(cellbox, CellBox):
        cellbox.add_splitting_condition(splitting_condition)
        
demo_mesh.split_to_depth(4)

In [ ]:
splitting_condition = '{"SIC":{"threshold":35,"upperBound": 0.9,"lowerBound":0.1}}'
splitting_condition = json.loads(splitting_condition)


for cellbox in demo_mesh.cellboxes:
    if isinstance(cellbox, CellBox):
        cellbox.add_splitting_condition(splitting_condition)
        
demo_mesh.split_to_depth(4)

# Generate output from cellbox creation

Saved as JSON

In [5]:
demo_output = demo_mesh.to_json()

NameError: name 'demo_mesh' is not defined

# Plot

Read in JSON output, plot each layer by type

In [ ]:
from bas_geoplot.interactive import Map
import pandas as pd


demo_mesh_df =pd.DataFrame(demo_output['cellboxes'])
mp = Map(title="demo mesh")
mp.Maps(demo_mesh_df, 'SIC', predefined='SIC')
mp.Vectors(demo_mesh_df, 'Currents', 3.0)
mp.MeshInfo(demo_mesh_df, 'Mesh Info', show=False)
# mp.Maps(demo_mesh_df, 'elevation', predefined = 'Elev')
mp.show()